# Initiate GPU runtime

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# Download sequencing run - Using Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

!ls gdrive/MyDrive

# Download sequencing run - Using MEGA.nz

In [ ]:
%%shell

sudo apt-get install -y megatools

# specify the MEGA url of the sequencing run (*.zip)
FILE=""

# specify the name of the file (e. g. IGM_PVM_MINION_LIG_LIBRARYyyyymmdd.zip)
FILE_NAME=""

megadl "$FILE" --path /content/
unzip "$FILE_NAME".zip; rm "$FILE_NAME".zip

# Install Dorado (Oxford Nanopore Technologies)

In [ ]:
%%shell

# https://nanoporetech.com/software/other/dorado
DORADO_VERSION=1.1.1

wget https://cdn.oxfordnanoportal.com/software/analysis/dorado-"$DORADO_VERSION"-linux-x64.tar.gz -O dorado.tar.gz
tar -vzxf dorado.tar.gz; rm -rf dorado.tar.gz; mv dorado-"$DORADO_VERSION"-linux-x64 ont-dorado; mkdir -p ont-dorado/models

./ont-dorado/bin/dorado download --directory ont-dorado/models

# BASECALL and DEMULTIPLEX process

In [ ]:
%%shell

# specify the name of the folder
FOLDER_NAME=""

# specify the basecalling model(s) accuracy(ies): fast,hac,sup
# https://nanoporetech.com/platform/accuracy
ACCURACY=sup

# specify the barcoding kit name.
# ./ont-dorado/bin/dorado demux --help
# EXP-NBD103 EXP-NBD104 EXP-NBD114 EXP-NBD114-24 EXP-NBD196 EXP-PBC001
# EXP-PBC096 SQK-16S024 SQK-16S114-24 SQK-DRB004-24 SQK-LWB001 SQK-MAB114-24
# SQK-MLK111-96-XL SQK-MLK114-96-XL SQK-NBD111-24 SQK-NBD111-96 SQK-NBD114-24
# SQK-NBD114-96 SQK-PBK004 SQK-PCB109 SQK-PCB110 SQK-PCB111-24 SQK-PCB114-24
# SQK-RAB201 SQK-RAB204 SQK-RBK001 SQK-RBK004 SQK-RBK110-96 SQK-RBK111-24
# SQK-RBK111-96 SQK-RBK114-24 SQK-RBK114-96 SQK-RLB001 SQK-RPB004 SQK-RPB114-24
# TWIST-16-UDI TWIST-96A-UDI VSK-PTC001 VSK-VMK001 VSK-VMK004 VSK-VPS001
BARCODE_KIT=SQK-RBK114-96

echo "$ACCURACY" | tr ',' '\n' | while read i; do
    ./ont-dorado/bin/dorado basecaller --models-directory /content/ont-dorado/models/ "$i" \
      /content/"$FOLDER_NAME"/ -r -v -x cuda:all --min-qscore 10 --no-trim \
      -o /content/"$FOLDER_NAME"-BASECALL-"$i"

    ./ont-dorado/bin/dorado summary /content/"$FOLDER_NAME"-BASECALL-"$i"/*/*/bam_pass/*.bam \
      >> /content/"$FOLDER_NAME"-BASECALL-"$i"/basecalling_summary_pass.txt

    ./ont-dorado/bin/dorado summary /content/"$FOLDER_NAME"-BASECALL-"$i"/*/*/bam_fail/*.bam \
      >> /content/"$FOLDER_NAME"-BASECALL-"$i"/basecalling_summary_fail.txt

    ./ont-dorado/bin/dorado demux /content/"$FOLDER_NAME"-BASECALL-"$i"/*/*/bam_pass/ \
    -t $(nproc) -r --kit-name "$BARCODE_KIT" --emit-summary \
    --output-dir /content/"$FOLDER_NAME"-DEMUX-"$i"

    tar -czvf "$FOLDER_NAME"-BASECALL-"$i".tar.gz "$FOLDER_NAME"-BASECALL-"$i"
    tar -czvf "$FOLDER_NAME"-DEMUX-"$i".tar.gz "$FOLDER_NAME"-DEMUX-"$i"
done

# Download the BASECALL and DEMUX analysis (*.taz.gz)

In [ ]:
from google.colab import files
import glob as gl

for i in gl.glob('*.tar.gz'):
    files.download(i)